In [134]:
# Import the dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from datetime import datetime
# To add time delays to the code
import time

In [135]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90, high = 90, size =2000)
lngs = np.random.uniform(low=-180, high = 180, size =2000)
lats_lngs = zip(lats, lngs)

In [136]:
coordinates = list(lats_lngs)
len(coordinates)

2000

In [137]:
# Create a list for holding the cities.
cities = []

for coordinate in coordinates:
    # Identify the nearest city for each latitude and longitude combination.
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
len(cities)

755

In [138]:
from config import weather_api_key
from config import g_key

In [139]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [140]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through all the cities in our list.
for i, city in enumerate(cities):   
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Group cities in sets of 50 to add time delay. This is to avoid api key blocking
    if (i % 50 == 0 and i >= 50):
        time.sleep(15)
    
   # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_current_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_current_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

            
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. 

In [141]:
len(city_data)

701

In [142]:
 # Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Boa Vista,2.8197,-60.6733,78.78,83,20,4.61,BR,few clouds
1,Hermanus,-34.4187,19.2345,52.90,75,0,11.18,ZA,clear sky
2,Vao,-22.6667,167.4833,73.45,71,95,3.22,NC,overcast clouds
3,Kirakira,-10.4544,161.9205,83.05,72,100,5.97,SB,overcast clouds
4,Oktyabrskoye,43.0645,44.7417,57.34,94,60,2.37,RU,broken clouds
5,Tasiilaq,65.6145,-37.6368,35.76,96,100,4.47,GL,light rain
6,Hobart,-42.8794,147.3294,58.05,60,75,7.00,AU,moderate rain
7,Wenling,28.3669,121.3600,80.91,91,100,1.95,CN,light rain
8,Ribeira Grande,38.5167,-28.7000,63.72,87,100,8.14,PT,overcast clouds
9,Deming,32.2687,-107.7586,80.49,14,1,5.75,US,clear sky


In [144]:
#re-order columns
new_column_order = ["City", "Country", "Lat", "Lng","Max Temp", "Humidity","Cloudiness","Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Boa Vista,BR,2.8197,-60.6733,78.78,83,20,4.61,few clouds
1,Hermanus,ZA,-34.4187,19.2345,52.90,75,0,11.18,clear sky
2,Vao,NC,-22.6667,167.4833,73.45,71,95,3.22,overcast clouds
3,Kirakira,SB,-10.4544,161.9205,83.05,72,100,5.97,overcast clouds
4,Oktyabrskoye,RU,43.0645,44.7417,57.34,94,60,2.37,broken clouds


In [146]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label = "City ID")